# Démonstration du Projet de Pricing et Gestion des Produits Dérivés de Taux

Ce notebook présente les principales fonctionnalités du projet, notamment :
- La modélisation des taux d'intérêt
- Le pricing des instruments dérivés de taux : swaps, caps, floors, swaptions
- Les stratégies de couverture
- L'analyse d'arbitrage

## 1. Configuration et imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

# Ajouter le répertoire parent au path pour l'import des modules
sys.path.append('..')

# Configuration des graphiques
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Import des modules du projet
from models.interest_rate import Vasicek, CIR
from models.derivatives import IRSwapPricer, CapFloorPricer, SwaptionPricer
from instruments.swaps import InterestRateSwap
from instruments.caps_floors import Cap, Floor, Collar
from instruments.swaptions import Swaption
from hedging.delta_hedge import DeltaHedger
from hedging.simulation import HedgingSimulator
from arbitrage.bond_pricing import BondPricer
from arbitrage.opportunities import ArbitrageAnalyzer

## 2. Modélisation des Taux d'Intérêt

Commençons par explorer les différents modèles de taux d'intérêt.

In [ ]:
# Paramètres des modèles
r0 = 0.03        # Taux initial
kappa = 0.5      # Vitesse de retour à la moyenne
theta = 0.05     # Niveau moyen à long terme
sigma = 0.01     # Volatilité
timesteps = 500  # Nombre de pas de temps
time_horizon = 10.0  # Horizon de 10 ans

# Création des modèles
vasicek_model = Vasicek(r0, kappa, theta, sigma, timesteps, time_horizon)
cir_model = CIR(r0, kappa, theta, sigma, timesteps, time_horizon)

In [ ]:
# Simulation des trajectoires
n_paths = 10
np.random.seed(42)  # Pour la reproductibilité

vasicek_rates = vasicek_model.simulate_rates(n_paths=n_paths)
cir_rates = cir_model.simulate_rates(n_paths=n_paths)

In [ ]:
# Visualisation des trajectoires
time = np.linspace(0, time_horizon, timesteps + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Vasicek
for i in range(n_paths):
    ax1.plot(time, vasicek_rates[i], alpha=0.7)
ax1.plot(time, np.mean(vasicek_rates, axis=0), 'k--', linewidth=2, label='Moyenne')
ax1.set_title('Modèle de Vasicek')
ax1.set_xlabel('Temps (années)')
ax1.set_ylabel('Taux d\'intérêt')
ax1.axhline(y=theta, color='r', linestyle=':', label=f'Niveau moyen θ={theta}')
ax1.legend()

# CIR
for i in range(n_paths):
    ax2.plot(time, cir_rates[i], alpha=0.7)
ax2.plot(time, np.mean(cir_rates, axis=0), 'k--', linewidth=2, label='Moyenne')
ax2.set_title('Modèle CIR (Cox-Ingersoll-Ross)')
ax2.set_xlabel('Temps (années)')
ax2.set_ylabel('Taux d\'intérêt')
ax2.axhline(y=theta, color='r', linestyle=':', label=f'Niveau moyen θ={theta}')
ax2.legend()

plt.tight_layout()

### Courbe des Taux Zéro-Coupon

In [ ]:
# Calcul des prix d'obligations zéro-coupon pour différentes maturités
maturities = np.linspace(0.25, 30, 50)  # Maturités de 3 mois à 30 ans
current_time = 0.0
current_rate = r0

# Prix des obligations zéro-coupon
vasicek_zcb_prices = [vasicek_model.zero_coupon_bond_price(current_time, T, current_rate) for T in maturities]
cir_zcb_prices = [cir_model.zero_coupon_bond_price(current_time, T, current_rate) for T in maturities]

# Calcul des taux zéro-coupon
vasicek_zero_rates = [-np.log(p) / m for p, m in zip(vasicek_zcb_prices, maturities)]
cir_zero_rates = [-np.log(p) / m for p, m in zip(cir_zcb_prices, maturities)]

In [ ]:
# Visualisation des courbes de taux zéro-coupon
plt.figure(figsize=(12, 6))
plt.plot(maturities, np.array(vasicek_zero_rates) * 100, label='Vasicek')
plt.plot(maturities, np.array(cir_zero_rates) * 100, label='CIR')
plt.title('Courbes de Taux Zéro-Coupon')
plt.xlabel('Maturité (années)')
plt.ylabel('Taux (%)')
plt.legend()
plt.grid(True)
plt.tight_layout()

## 3. Pricing des Swaps de Taux

In [ ]:
# Création du pricer de swap
swap_pricer = IRSwapPricer(vasicek_model)

# Paramètres du swap
start_date = 0.0     # Aujourd'hui
maturity_date = 10.0  # Swap à 10 ans
payment_frequency = 0.5  # Paiements semestriels

In [ ]:
# Calcul du taux de swap à la parité pour différentes maturités
maturities = np.linspace(1, 30, 30)  # 1 an à 30 ans
par_rates = [swap_pricer.par_rate(start_date, maturity, payment_frequency) for maturity in maturities]

# Visualisation de la courbe des taux de swap
plt.figure(figsize=(12, 6))
plt.plot(maturities, np.array(par_rates) * 100)
plt.title('Courbe des Taux de Swap')
plt.xlabel('Maturité (années)')
plt.ylabel('Taux de Swap (%)')
plt.grid(True)
plt.tight_layout()

In [ ]:
# Calcul du taux de swap à la parité pour un swap de 10 ans
par_rate_10y = swap_pricer.par_rate(start_date, maturity_date, payment_frequency)
print(f"Taux de swap à la parité pour un swap de 10 ans: {par_rate_10y:.4%}")

# Création d'un swap avec un taux fixe différent du taux à la parité
fixed_rate = 0.04  # 4%
notional = 1000000  # 1 million

# Création d'un swap payeur et d'un swap receveur
payer_swap = InterestRateSwap(
    start_date=start_date,
    maturity_date=maturity_date,
    fixed_rate=fixed_rate,
    payment_frequency=payment_frequency,
    notional=notional,
    is_payer=True  # Payeur de taux fixe
)

receiver_swap = InterestRateSwap(
    start_date=start_date,
    maturity_date=maturity_date,
    fixed_rate=fixed_rate,
    payment_frequency=payment_frequency,
    notional=notional,
    is_payer=False  # Receveur de taux fixe
)

# Évaluation des swaps
payer_swap_npv = payer_swap.price(swap_pricer)
receiver_swap_npv = receiver_swap.price(swap_pricer)

print(f"NPV du swap payeur: {payer_swap_npv:,.2f}")
print(f"NPV du swap receveur: {receiver_swap_npv:,.2f}")

In [ ]:
# Analyse de sensibilité au taux d'intérêt
rates = np.linspace(0.01, 0.10, 50)  # Taux de 1% à 10%
payer_npvs = []
receiver_npvs = []

for rate in rates:
    # Mise à jour du taux dans le modèle
    vasicek_model.r0 = rate
    
    # Calcul des NPV avec le nouveau taux
    payer_npv = payer_swap.price(swap_pricer)
    receiver_npv = receiver_swap.price(swap_pricer)
    
    payer_npvs.append(payer_npv)
    receiver_npvs.append(receiver_npv)

# Restauration du taux initial
vasicek_model.r0 = r0

# Affichage du graphique de sensibilité
plt.figure(figsize=(12, 6))
plt.plot(rates * 100, payer_npvs, label='Swap Payeur')
plt.plot(rates * 100, receiver_npvs, label='Swap Receveur')
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.axvline(x=fixed_rate * 100, color='g', linestyle='--', alpha=0.7, label=f'Taux fixe ({fixed_rate:.2%})')
plt.title('Sensibilité du NPV du swap au taux d\'intérêt')
plt.xlabel('Taux d\'intérêt (%)')
plt.ylabel('NPV du swap')
plt.legend()
plt.grid(True)
plt.tight_layout()

## 4. Pricing des Caps, Floors et Collars

In [ ]:
# Création du pricer pour les caps et floors
capfloor_pricer = CapFloorPricer(vasicek_model)

# Paramètres des options
start_date = 0.0      # Aujourd'hui
maturity_date = 5.0   # 5 ans
cap_strike = 0.045    # Strike du Cap à 4.5%
floor_strike = 0.025  # Strike du Floor à 2.5%
payment_frequency = 0.5  # Paiements semestriels
notional = 1000000    # 1 million
volatility = 0.015    # Volatilité de 1.5%

In [ ]:
# Création des instruments
cap = Cap(
    start_date=start_date,
    maturity_date=maturity_date,
    strike=cap_strike,
    payment_frequency=payment_frequency,
    notional=notional
)

floor = Floor(
    start_date=start_date,
    maturity_date=maturity_date,
    strike=floor_strike,
    payment_frequency=payment_frequency,
    notional=notional
)

collar = Collar(
    start_date=start_date,
    maturity_date=maturity_date,
    cap_strike=cap_strike,
    floor_strike=floor_strike,
    payment_frequency=payment_frequency,
    notional=notional
)

# Pricing des instruments
cap_price = cap.price(capfloor_pricer, volatility=volatility)
floor_price = floor.price(capfloor_pricer, volatility=volatility)
collar_price = collar.price(capfloor_pricer, volatility=volatility)

print(f"Prix du Cap (Strike = {cap_strike:.2%}): {cap_price:,.2f}")
print(f"Prix du Floor (Strike = {floor_strike:.2%}): {floor_price:,.2f}")
print(f"Prix du Collar (Cap = {cap_strike:.2%}, Floor = {floor_strike:.2%}): {collar_price:,.2f}")

In [ ]:
# Analyse de sensibilité à la volatilité
volatilities = np.linspace(0.005, 0.03, 20)  # Volatilités de 0.5% à 3%
cap_prices = []
floor_prices = []
collar_prices = []

for vol in volatilities:
    cap_price = cap.price(capfloor_pricer, volatility=vol)
    floor_price = floor.price(capfloor_pricer, volatility=vol)
    collar_price = collar.price(capfloor_pricer, volatility=vol)
    
    cap_prices.append(cap_price)
    floor_prices.append(floor_price)
    collar_prices.append(collar_price)

# Graphique de sensibilité à la volatilité
plt.figure(figsize=(12, 6))
plt.plot(volatilities * 100, cap_prices, label='Cap')
plt.plot(volatilities * 100, floor_prices, label='Floor')
plt.plot(volatilities * 100, collar_prices, label='Collar')
plt.title('Sensibilité des prix à la volatilité')
plt.xlabel('Volatilité (%)')
plt.ylabel('Prix')
plt.legend()
plt.grid(True)
plt.tight_layout()

In [ ]:
# Analyse de sensibilité au taux sous-jacent
rates = np.linspace(0.01, 0.07, 30)  # Taux de 1% à 7%
cap_prices = []
floor_prices = []
collar_prices = []

for rate in rates:
    # Mise à jour du taux dans le modèle
    vasicek_model.r0 = rate
    
    cap_price = cap.price(capfloor_pricer, volatility=volatility)
    floor_price = floor.price(capfloor_pricer, volatility=volatility)
    collar_price = collar.price(capfloor_pricer, volatility=volatility)
    
    cap_prices.append(cap_price)
    floor_prices.append(floor_price)
    collar_prices.append(collar_price)

# Restauration du taux initial
vasicek_model.r0 = r0

# Graphique de sensibilité au taux sous-jacent
plt.figure(figsize=(12, 6))
plt.plot(rates * 100, cap_prices, label='Cap')
plt.plot(rates * 100, floor_prices, label='Floor')
plt.plot(rates * 100, collar_prices, label='Collar')
plt.axvline(x=cap_strike * 100, color='r', linestyle='--', alpha=0.5, label=f'Cap Strike ({cap_strike:.2%})')
plt.axvline(x=floor_strike * 100, color='g', linestyle='--', alpha=0.5, label=f'Floor Strike ({floor_strike:.2%})')
plt.title('Sensibilité des prix au taux sous-jacent')
plt.xlabel('Taux sous-jacent (%)')
plt.ylabel('Prix')
plt.legend()
plt.grid(True)
plt.tight_layout()

## 5. Pricing des Swaptions

In [ ]:
# Création du pricer pour les swaptions
swaption_pricer = SwaptionPricer(vasicek_model, swap_pricer)

# Paramètres de la swaption
expiry_date = 2.0  # Expiration dans 2 ans
underlying_swap_maturity = 5.0  # Swap sous-jacent de 5 ans
volatility = 0.015  # Volatilité de 1.5%

# Calcul du taux de swap forward pour le swap sous-jacent
forward_swap_rate = swap_pricer.par_rate(expiry_date, expiry_date + underlying_swap_maturity, payment_frequency)
print(f"Taux de swap forward à {expiry_date} ans: {forward_swap_rate:.4%}")

# Création d'une swaption avec un strike proche du forward rate
swaption_strike = round(forward_swap_rate * 100) / 100  # Arrondi pour simplifier

payer_swaption = Swaption(
    expiry_date=expiry_date,
    underlying_swap_maturity=underlying_swap_maturity,
    strike_rate=swaption_strike,
    is_payer=True,  # Swaption payeuse
    payment_frequency=payment_frequency,
    notional=notional
)

receiver_swaption = Swaption(
    expiry_date=expiry_date,
    underlying_swap_maturity=underlying_swap_maturity,
    strike_rate=swaption_strike,
    is_payer=False,  # Swaption receveuse
    payment_frequency=payment_frequency,
    notional=notional
)

# Pricing des swaptions
payer_price = payer_swaption.price(swaption_pricer, volatility=volatility)
receiver_price = receiver_swaption.price(swaption_pricer, volatility=volatility)

print(f"Prix de la Swaption Payeuse (Strike = {swaption_strike:.2%}): {payer_price:,.2f}")
print(f"Prix de la Swaption Receveuse (Strike = {swaption_strike:.2%}): {receiver_price:,.2f}")

In [ ]:
# Analyse de sensibilité à la volatilité
volatilities = np.linspace(0.005, 0.03, 20)  # Volatilités de 0.5% à 3%
payer_prices = []
receiver_prices = []

for vol in volatilities:
    payer_price = payer_swaption.price(swaption_pricer, volatility=vol)
    receiver_price = receiver_swaption.price(swaption_pricer, volatility=vol)
    
    payer_prices.append(payer_price)
    receiver_prices.append(receiver_price)

# Graphique de sensibilité à la volatilité
plt.figure(figsize=(12, 6))
plt.plot(volatilities * 100, payer_prices, label='Swaption Payeuse')
plt.plot(volatilities * 100, receiver_prices, label='Swaption Receveuse')
plt.title('Sensibilité des prix de swaption à la volatilité')
plt.xlabel('Volatilité (%)')
plt.ylabel('Prix')
plt.legend()
plt.grid(True)
plt.tight_layout()

## 6. Couverture Dynamique (Delta Hedging)

In [ ]:
# Création d'une stratégie de couverture pour un cap
cap_hedger = DeltaHedger(
    rate_model=vasicek_model,
    derivative=cap,
    rebalance_frequency=0.1  # Rééquilibrage tous les 1,2 mois (0.1 ans)
)

# Simulation de la couverture
hedge_results = cap_hedger.simulate_hedging(
    time_horizon=5.0,  # Simulation sur 5 ans
    n_paths=50,        # 50 trajectoires
    seed=42            # Pour la reproductibilité
)

In [ ]:
# Affichage des trajectoires de PnL
plt.figure(figsize=(12, 6))

# Affichage de quelques trajectoires
n_displayed = 10
sample_indices = np.random.choice(hedge_results['pnl'].shape[0], n_displayed, replace=False)

for idx in sample_indices:
    plt.plot(hedge_results['times'], hedge_results['pnl'][idx], alpha=0.5)

# Affichage du PnL moyen
plt.plot(hedge_results['times'], np.mean(hedge_results['pnl'], axis=0), 'k--', linewidth=2, label='PnL moyen')

plt.title('Profit & Loss (PnL) de la stratégie de couverture')
plt.xlabel('Temps (années)')
plt.ylabel('PnL')
plt.grid(True)
plt.legend()
plt.tight_layout()

In [ ]:
# Affichage de la distribution du PnL final
plt.figure(figsize=(10, 6))
final_pnl = hedge_results['pnl'][:, -1]

plt.hist(final_pnl, bins=15, alpha=0.7)
plt.axvline(x=np.mean(final_pnl), color='r', linestyle='--', 
            label=f'Moyenne: {np.mean(final_pnl):.2f}')
plt.axvline(x=np.percentile(final_pnl, 5), color='g', linestyle=':', 
            label=f'5% VaR: {np.percentile(final_pnl, 5):.2f}')

plt.title('Distribution du PnL final')
plt.xlabel('PnL final')
plt.ylabel('Fréquence')
plt.legend()
plt.grid(True)
plt.tight_layout()

In [ ]:
# Analyse des ratios de couverture
plt.figure(figsize=(12, 6))

for idx in sample_indices:
    plt.plot(hedge_results['times'], hedge_results['hedge_ratios'][idx], alpha=0.5)

# Affichage du ratio de couverture moyen
plt.plot(hedge_results['times'], np.mean(hedge_results['hedge_ratios'], axis=0), 'k--', linewidth=2, label='Ratio moyen')

plt.title('Évolution des ratios de couverture (Delta)')
plt.xlabel('Temps (années)')
plt.ylabel('Delta')
plt.grid(True)
plt.legend()
plt.tight_layout()

## 7. Analyse d'Arbitrage entre Obligations et Dérivés de Taux

In [ ]:
# Création des pricers
bond_pricer = BondPricer(vasicek_model)

# Création de l'analyseur d'arbitrage
arbitrage_analyzer = ArbitrageAnalyzer(
    bond_pricer=bond_pricer,
    swap_pricer=swap_pricer,
    option_pricer=capfloor_pricer
)

### 7.1 Analyse d'Obligations vs Swaps

In [ ]:
# Paramètres de l'obligation
bond_maturity = 5.0    # Maturité de 5 ans
bond_coupon = 0.04     # Coupon de 4%

# Prix théorique
bond_price_theoretical = bond_pricer.price_fixed_coupon_bond(
    maturity=bond_maturity,
    coupon_rate=bond_coupon
)

print(f"Prix théorique de l'obligation: {bond_price_theoretical:.4f}")

# Scénarios de prix observés
scenarios = {
    "Sous-évalué (-2%)": bond_price_theoretical * 0.98,
    "Prix théorique": bond_price_theoretical,
    "Surévalué (+2%)": bond_price_theoretical * 1.02
}

# Analyse pour chaque scénario
for scenario_name, bond_price in scenarios.items():
    print(f"\n{'-'*50}\nScénario: {scenario_name}")
    print(f"Prix observé de l'obligation: {bond_price:.4f}")
    
    # Analyse d'arbitrage
    result = arbitrage_analyzer.analyze_bond_vs_swaps(
        bond_maturity=bond_maturity,
        bond_coupon_rate=bond_coupon,
        bond_price=bond_price,
        transaction_costs=0.001  # 0.1% de coûts de transaction
    )
    
    # Affichage des résultats
    print(f"YTM de l'obligation: {result['bond_ytm']:.4%}")
    print(f"Taux de swap: {result['swap_rate']:.4%}")
    print(f"Spread: {result['spread']:.4%}")
    print(f"Opportunité d'arbitrage: {'Oui' if result['arbitrage_opportunity'] else 'Non'}")
    if result['arbitrage_opportunity']:
        print(f"Stratégie: {result['strategy']}")
        print(f"Profit estimé (après coûts): {result['estimated_profit_after_costs']:.4f} par unité notionnelle")

### 7.2 Analyse d'Asset Swap

In [ ]:
# Paramètres de l'obligation
bond_maturity = 5.0    # Maturité de 5 ans
bond_coupon = 0.05     # Coupon de 5% (plus élevé que le taux de swap)

# Prix théorique
bond_price_theoretical = bond_pricer.price_fixed_coupon_bond(
    maturity=bond_maturity,
    coupon_rate=bond_coupon
)

# Scénario de prix avec décote
bond_price_discounted = bond_price_theoretical * 0.97  # Obligation avec décote (-3%)

print(f"Prix théorique de l'obligation: {bond_price_theoretical:.4f}")
print(f"Prix observé avec décote (-3%): {bond_price_discounted:.4f}")

# Analyse d'asset swap
result = arbitrage_analyzer.analyze_asset_swap(
    bond_maturity=bond_maturity,
    bond_coupon_rate=bond_coupon,
    bond_price=bond_price_discounted,
    transaction_costs=0.001  # 0.1% de coûts de transaction
)

# Affichage des résultats
print(f"\nRésultats de l'analyse d'asset swap:")
print(f"YTM de l'obligation: {result['bond_ytm']:.4%}")
print(f"Taux de swap: {result['swap_rate']:.4%}")
print(f"Spread d'asset swap: {result['asset_swap_spread']:.4%}")
print(f"Opportunité d'arbitrage: {'Oui' if result['arbitrage_opportunity'] else 'Non'}")
if result['arbitrage_opportunity']:
    print(f"Stratégie: {result['strategy']}")
    print(f"Profit estimé (après coûts): {result['estimated_profit_after_costs']:.4f} par unité notionnelle")

### 7.3 Analyse d'Obligation vs Cap/Floor

In [ ]:
# Paramètres
bond_maturity = 5.0    # Maturité de 5 ans
bond_coupon = 0.04     # Coupon de 4%
cap_strike = 0.045     # Strike du Cap à 4.5%
floor_strike = 0.025   # Strike du Floor à 2.5%
vol = 0.015            # Volatilité de 1.5%

# Prix théorique de l'obligation
bond_price_theoretical = bond_pricer.price_fixed_coupon_bond(
    maturity=bond_maturity,
    coupon_rate=bond_coupon
)

print(f"Prix théorique de l'obligation: {bond_price_theoretical:.4f}")

# Analyse avec les options
result = arbitrage_analyzer.analyze_bond_vs_capfloor(
    bond_maturity=bond_maturity,
    bond_coupon_rate=bond_coupon,
    cap_strike=cap_strike,
    floor_strike=floor_strike,
    bond_price=bond_price_theoretical,
    volatility=vol,
    transaction_costs=0.002  # 0.2% de coûts de transaction
)

# Affichage des résultats
print(f"\nRésultats de l'analyse d'arbitrage avec options:")
print(f"YTM de l'obligation: {result['bond_ytm']:.4%}")
print(f"Taux forward moyen: {result['avg_forward_rate']:.4%}")
print(f"\nPrix du Cap (Strike = {cap_strike:.2%}): {result['cap_price']:.4f}")
print(f"Stratégie (Cap): {result['cap_strategy']}")
print(f"\nPrix du Floor (Strike = {floor_strike:.2%}): {result['floor_price']:.4f}")
print(f"Stratégie (Floor): {result['floor_strategy']}")
print(f"\nPrix du Collar: {result['collar_price']:.4f}")
print(f"Stratégie (Collar): {result['collar_strategy']}")

### 7.4. Analyse de l'impact du spread de crédit

In [ ]:
# Paramètres de l'obligation
bond_maturity = 7.0    # Maturité de 7 ans
bond_coupon = 0.045    # Coupon de 4.5%

# Spreads de crédit à analyser
credit_spreads = np.linspace(0.0, 0.02, 20)  # 0% à 2%

# Résultats pour différents spreads
bond_prices = []
bond_ytms = []
swap_rates = []
asset_swap_spreads = []

for spread in credit_spreads:
    # Pricer avec spread de crédit
    bond_pricer_with_spread = BondPricer(vasicek_model, credit_spread=spread)
    
    # Prix de l'obligation avec spread
    bond_price = bond_pricer_with_spread.price_fixed_coupon_bond(
        maturity=bond_maturity,
        coupon_rate=bond_coupon
    )
    
    # YTM de l'obligation
    bond_ytm = bond_pricer.calculate_yield_to_maturity(
        bond_price=bond_price,
        maturity=bond_maturity,
        coupon_rate=bond_coupon
    )
    
    # Taux de swap
    swap_rate = swap_pricer.par_rate(0, bond_maturity)
    
    # Spread d'asset swap
    adjusted_coupon = bond_coupon * 100 / bond_price
    asset_swap_spread = adjusted_coupon - swap_rate
    
    # Stockage des résultats
    bond_prices.append(bond_price)
    bond_ytms.append(bond_ytm)
    swap_rates.append(swap_rate)
    asset_swap_spreads.append(asset_swap_spread)

In [ ]:
# Graphique des résultats
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Prix et YTM
ax1.plot(credit_spreads * 100, bond_prices, label='Prix de l\'obligation')
ax1_twin = ax1.twinx()
ax1_twin.plot(credit_spreads * 100, np.array(bond_ytms) * 100, 'r--', label='YTM')
ax1_twin.plot(credit_spreads * 100, np.array(swap_rates) * 100, 'g--', label='Taux de swap')

ax1.set_ylabel('Prix de l\'obligation')
ax1_twin.set_ylabel('Taux (%)')
ax1.set_title('Impact du spread de crédit sur le prix de l\'obligation et les taux')

# Légende combinée
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twin.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Spread d'asset swap
ax2.plot(credit_spreads * 100, np.array(asset_swap_spreads) * 100, 'b-', label='Spread d\'asset swap')
ax2.axhline(y=0, color='r', linestyle='--', alpha=0.5)

ax2.set_xlabel('Spread de crédit (%)')
ax2.set_ylabel('Spread d\'asset swap (%)')
ax2.set_title('Impact du spread de crédit sur le spread d\'asset swap')
ax2.legend()

plt.tight_layout()

## Conclusion

Ce notebook a démontré les principales fonctionnalités de notre bibliothèque de pricing et gestion des produits dérivés de taux. Nous avons exploré :

1. **Modélisation des taux d'intérêt** : Simulation des trajectoires et courbes de taux avec les modèles de Vasicek et CIR
2. **Pricing des instruments dérivés** : Évaluation des swaps, caps, floors, collars et swaptions
3. **Analyse de sensibilité** : Impact des variations de taux et de volatilité sur les prix des instruments
4. **Couverture dynamique** : Simulation de stratégies de delta-hedging pour un cap
5. **Analyse d'arbitrage** : Détection d'opportunités entre obligations et dérivés de taux

Cette bibliothèque offre une base solide pour l'analyse, le pricing et la gestion des risques liés aux produits dérivés de taux d'intérêt, pouvant être utilisée tant pour des fins pédagogiques que pour des applications pratiques.